Multi-criteria decision making (MCDM) using Topsis

In [1]:
# import nbformat to handle notebook files to call AHP notebook with %run ../AHP/AHP.ipynb
# pip install nbformat

In [2]:
# Imports

# Executar o notebook AHP para carregar os pesos
%run ../../AHP/test/AHP.ipynb

%run matrix_for_TOPSIS.ipynb
mydata # <- Dataframe importado com os dados
print(mydata)


# Estrutura do DataFrame necessária para o funcionamento do TOPSIS:
# |─────────────|───────────|─────────────────|───────────────────|─────────────────|────────────|─────────────────────────|
# │ Donor_id    │ HLA Match │ CMV Serostatus  │  Donor Age Group  │ Gender Match    │ ABO Match  │ Expected Survival Time  │
# ├─────────────┼───────────┼─────────────────┼───────────────────┼─────────────────┼────────────┼─────────────────────────┤
# │ str         │ int       │ int             │ int               │ int             │ int        │ int                     │
# └─────────────┴───────────┴─────────────────┴───────────────────┴─────────────────┴────────────┴─────────────────────────┘

import numpy as np
import pandas as pd

Pesos AHP:
{'HLA': np.float64(0.4029), 'Idade_dador': np.float64(0.3088), 'CMV': np.float64(0.1423), 'ABO': np.float64(0.0604), 'Sexo': np.float64(0.0555), 'Survival_Time': np.float64(0.0302)}

Consistency Ratio (CR):
0.0277
    Donor_id  HLA Match  CMV Serostatus  Donor Age Group  Gender Match  \
0          1          7               0                0             0   
1          2          8               1                1             1   
2          3          9               2                2             0   
3          4          7               3                0             1   
4          5         10               0                2             0   
5          6         10               3                0             1   
6          7          8               0                1             1   
7          8          9               1                0             0   
8          9          7               2                2             1   
9         10         10            

In [3]:
# Drop the first column (ids) and convert to NumPy array
matrix = mydata.iloc[:, 1:].values.astype(int)
matrix

array([[   7,    0,    0,    0,    0,  100],
       [   8,    1,    1,    1,    1,  200],
       [   9,    2,    2,    0,    0,  300],
       [   7,    3,    0,    1,    0,   50],
       [  10,    0,    2,    0,    1,  500],
       [  10,    3,    0,    1,    0,   40],
       [   8,    0,    1,    1,    0,  220],
       [   9,    1,    0,    0,    1,  340],
       [   7,    2,    2,    1,    0,   90],
       [  10,    0,    1,    0,    1,  480],
       [   9,    3,    0,    1,    1,  310],
       [   8,    1,    2,    0,    0,  180],
       [  10,    2,    0,    1,    1, 1020],
       [   7,    0,    1,    0,    0,  110],
       [   9,    1,    1,    1,    0,  290],
       [   8,    3,    2,    0,    1,  160],
       [  10,    0,    0,    1,    0,  550],
       [   7,    2,    1,    0,    1,  130],
       [   9,    0,    2,    1,    1,  260],
       [   8,    1,    0,    0,    0,  210],
       [  10,    3,    1,    1,    1,  470],
       [   7,    1,    2,    0,    0,   95],
       [  

In [4]:
# Assigning the impacts
# 1 for maximization and -1 for minimization
# Maximize: HLA_match, donor_age_group, ABO_match, expected_time_survival, Gender_match
# Minimize: CMV_status

criteria_preferences = np.array([1, -1, 1, 1, 1, 1], dtype=int)
criteria_preferences

array([ 1, -1,  1,  1,  1,  1])

In [5]:
# Assigning the weights
# # Os pesos foram definidos usando o método de apoio à decisão multicritério AHP (Analytic Hierarchy Process)


# Os pesos abaixo são importados do notebook AHP
criteria_weight = np.array([peso_HLA, peso_CMV, peso_Idade_dador, peso_Sexo, peso_ABO, peso_Survival_Time], dtype=float)
criteria_weight

array([0.4029, 0.1423, 0.3088, 0.0555, 0.0604, 0.0302])

In [6]:
# Euclidean normalization

def normalize_matrix(matrix):
    matrix_rows = len(matrix)
    matrix_columns = len(matrix[0])
    column_sums= [0] * matrix_columns

    for j in range(matrix_columns):
        for i in range(matrix_rows):
            column_sums[j] += matrix[i][j] ** 2
    column_sums = [value ** 0.5 for value in column_sums]

    normalized_matrix = []
    for i in range(matrix_rows):
        normalized_matrix_rows = []
        for j in range(matrix_columns):
            normalized_matrix_rows.append(matrix[i][j]/column_sums[j])
        normalized_matrix.append(normalized_matrix_rows)
    
    print('\nNormalized matrix from original matrix: ')
    for i in normalized_matrix:
        for j in i:
            print(f'{j:.4f}',end=' ')
        print()
    return normalized_matrix

# normalize_matrix(matrix)

In [7]:
# New matrix with weights applied

def weighted_matrix(criteria_weight, normalized_matrix):
    normalized_rows = len(normalized_matrix)
    normalized_columns = len(normalized_matrix[0])
    
    weighted_matrix = []
    for i in range(normalized_rows):
        weighted_matrix_rows = []
        for j in range(normalized_columns):
            weighted_matrix_rows.append(normalized_matrix [i][j]* criteria_weight[j])
        weighted_matrix.append(weighted_matrix_rows)
    
    print('\nWeighted matrix from normalized matrix: ')
    for i in weighted_matrix:
        for j in i:
            print(f'{j:.4f}',end=' ')
        print()
    return weighted_matrix

# weighted_matrix(criteria_weight, normalize_matrix(matrix))

In [8]:
# Positive ideal solution and negative ideal solution

def ideal_best_worst(weighted_matrix,criteria_preferences):
    weighted_column = len(weighted_matrix[0])
    positive_ideal= [] 
    negative_ideal = [] 

    for j in range(weighted_column):
        max_value = weighted_matrix[0][j]
        min_value = weighted_matrix[0][j]

        for i in range(len(weighted_matrix)):
            if weighted_matrix[i][j] > max_value:
                max_value = weighted_matrix [i][j]
            if weighted_matrix[i][j] < min_value:
                min_value = weighted_matrix [i][j]
        if criteria_preferences[j] == 1:  
            positive_ideal.append(max_value)
            negative_ideal.append(min_value)
        else:  
            positive_ideal.append(min_value)
            negative_ideal.append(max_value)
    
    print('\nPositive ideal point for each column: ')
    for i in positive_ideal:
        print(f'{i:.4f}',end=' ')
    print()
    print('\nNegative ideal point for each column: ')
    for i in negative_ideal:
        print(f'{i:.4f}',end=' ')
    print()

    return positive_ideal, negative_ideal

#ideal_best_worst(weighted_matrix(criteria_weight, normalize_matrix(matrix)), criteria_preferences)

In [9]:
# Calculation of the Separation Measures

def separation_from_ideal_point(weighted_matrix,positive_ideal,negative_ideal):
    weighted_rows = len(weighted_matrix)
    positive_separation = []
    negative_separation = []

    for i in range(weighted_rows):
        pos_sep = 0
        neg_sep = 0
        for j in range(len(positive_ideal)):
            pos_sep += (weighted_matrix[i][j] - positive_ideal[j]) ** 2
            neg_sep += (weighted_matrix[i][j] - negative_ideal[j]) ** 2
        positive_separation.append(pos_sep ** 0.5)
        negative_separation.append(neg_sep ** 0.5)

    print('\nPositive separation: ')
    for i in (positive_separation):
        print(f'{i:.4f}')
    print('\nNegative separation: ')
    for i in (negative_separation):
        print(f'{i:.4f}')
    return positive_separation,negative_separation

#separation_from_ideal_point(weighted_matrix(criteria_weight, normalize_matrix(matrix)),ideal_best_worst(weighted_matrix(criteria_weight, normalize_matrix(matrix)), criteria_preferences)[0],ideal_best_worst(weighted_matrix(criteria_weight, normalize_matrix(matrix)), criteria_preferences)[1])

In [10]:
# Relative closeness to the Ideal Solution

def similarities_to_PIS(positive_separation,negative_separation):
    num_rows = len(positive_separation)
    relative_similarity = []

    for i in range(num_rows):
        pos_sep = positive_separation[i]
        neg_sep = negative_separation[i]
        similarity = neg_sep/(pos_sep + neg_sep)
        relative_similarity.append(similarity)
    
    print('\nOrder: ')
    for i in (relative_similarity):
        print(f'{i:.4f}')
    return relative_similarity

#similarities_to_PIS(separation_from_ideal_point(weighted_matrix(criteria_weight, normalize_matrix(matrix)),ideal_best_worst(weighted_matrix(criteria_weight, normalize_matrix(matrix)), criteria_preferences)[0],ideal_best_worst(weighted_matrix(criteria_weight, normalize_matrix(matrix)), criteria_preferences)[1])[0],separation_from_ideal_point(weighted_matrix(criteria_weight, normalize_matrix(matrix)),ideal_best_worst(weighted_matrix(criteria_weight, normalize_matrix(matrix)), criteria_preferences)[0],ideal_best_worst(weighted_matrix(criteria_weight, normalize_matrix(matrix)), criteria_preferences)[1])[1])

In [11]:
# Cria tabela final

# Get the first column
first_column = mydata.iloc[:, 0]  # Keep as Series

# Convert results to Series
results_series = pd.Series(similarities_to_PIS(separation_from_ideal_point(weighted_matrix(criteria_weight, normalize_matrix(matrix)),ideal_best_worst(weighted_matrix(criteria_weight, normalize_matrix(matrix)), criteria_preferences)[0],ideal_best_worst(weighted_matrix(criteria_weight, normalize_matrix(matrix)), criteria_preferences)[1])[0],separation_from_ideal_point(weighted_matrix(criteria_weight, normalize_matrix(matrix)),ideal_best_worst(weighted_matrix(criteria_weight, normalize_matrix(matrix)), criteria_preferences)[0],ideal_best_worst(weighted_matrix(criteria_weight, normalize_matrix(matrix)), criteria_preferences)[1])[1]), name='TOPSIS Score')

# Concatenate along columns
df_TOPSIS = pd.concat([first_column, results_series], axis=1)
#print(df_TOPSIS)

# Sort by TOPSIS_Score in descending order (highest to lowest)
df_TOPSIS = df_TOPSIS.sort_values(by='TOPSIS Score', ascending=False)
df_TOPSIS.rename(columns={'TOPSIS Score': 'TOPSIS Rank'}, inplace=True)

df_TOPSIS # <- Tabela a ser retornada


Normalized matrix from original matrix: 
0.1621 0.0000 0.0000 0.0000 0.0000 0.0567 
0.1853 0.1179 0.1581 0.2774 0.2887 0.1134 
0.2085 0.2357 0.3162 0.0000 0.0000 0.1701 
0.1621 0.3536 0.0000 0.2774 0.0000 0.0283 
0.2316 0.0000 0.3162 0.0000 0.2887 0.2835 
0.2316 0.3536 0.0000 0.2774 0.0000 0.0227 
0.1853 0.0000 0.1581 0.2774 0.0000 0.1247 
0.2085 0.1179 0.0000 0.0000 0.2887 0.1928 
0.1621 0.2357 0.3162 0.2774 0.0000 0.0510 
0.2316 0.0000 0.1581 0.0000 0.2887 0.2721 
0.2085 0.3536 0.0000 0.2774 0.2887 0.1758 
0.1853 0.1179 0.3162 0.0000 0.0000 0.1021 
0.2316 0.2357 0.0000 0.2774 0.2887 0.5783 
0.1621 0.0000 0.1581 0.0000 0.0000 0.0624 
0.2085 0.1179 0.1581 0.2774 0.0000 0.1644 
0.1853 0.3536 0.3162 0.0000 0.2887 0.0907 
0.2316 0.0000 0.0000 0.2774 0.0000 0.3118 
0.1621 0.2357 0.1581 0.0000 0.2887 0.0737 
0.2085 0.0000 0.3162 0.2774 0.2887 0.1474 
0.1853 0.1179 0.0000 0.0000 0.0000 0.1191 
0.2316 0.3536 0.1581 0.2774 0.2887 0.2665 
0.1621 0.1179 0.3162 0.0000 0.0000 0.0539 
0.2085 0.235

,Donor_id,TOPSIS Rank
18,19,0.876726
4,5,0.866041
24,25,0.814315
11,12,0.736434
21,22,0.705554
2,3,0.698092
8,9,0.668776
15,16,0.633173
9,10,0.599413
23,24,0.580091


In [12]:
# Função para ser chamada por outros notebooks
def get_topsis_results():
    """
    Retorna a tabela TOPSIS com os rankings dos doadores
    """
    return df_TOPSIS